In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ee
import geemap
import geopandas as gpd
import contextily as ctx
from datetime import datetime, timedelta

In [2]:
#Authenticate with Earth Engine
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
#Load the Lake Victoria shapefile Asset
lake = "projects/water-quality-441207/assets/lake_victoria"
lake = ee.FeatureCollection(lake)

In [5]:
#Create map
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
#add the lake as layer
m.addLayer(lake, {}, 'Lake Victoria')
m.centerObject(lake, 7)

In [11]:
# Define a function to mask clouds in Sentinel-2 images
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

# Load and preprocess Sentinel-2 dataset
s2_dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2017-03-28', '2024-11-12')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# Calculate NDWI
ndwi_bands = ['B3', 'B8']
ndwi = s2_dataset.map(lambda image: image.normalizedDifference(ndwi_bands).rename('ndwi'))

# Calculate NDCI
ndci_bands = ['B5', 'B4']
ndci = s2_dataset.map(lambda image: image.normalizedDifference(ndci_bands).rename('ndci'))

# Calculate NDTI
ndti_bands = ['B5', 'B11']
ndti = s2_dataset.map(lambda image: image.normalizedDifference(ndti_bands).rename('ndti'))

# Calculate pH
ph = s2_dataset.map(lambda image: ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('ph'))

# Calculate chlorophyll-a (chl_a)
chl_a = s2_dataset.map(lambda image: image.select('B3').divide(0.1).divide(image.select('B2').divide(0.1)).pow(2.72).multiply(10.8).rename('chl_a'))

# Calculate SSC
ssc = s2_dataset.map(lambda image: ee.Image(0.0113).multiply(image.select('ndwi').pow(3))
                    .subtract(ee.Image(0.0135).multiply(image.select('ndwi').pow(2)))
                    .add(ee.Image(0.0075).multiply(image.select('ndwi')))
                    .add(ee.Image(2.5823)).rename('ssc'))

# Calculate Water Surface Temperature (WST)
tir2 = s2_dataset.select('B11')
kelvin = tir2.map(lambda image: image.divide(10).add(273.15))
lst = kelvin.map(lambda image: image.divide(ee.Image(1).toFloat().divide(image.divide(14380).add(1).log())))
emissivity = ee.Image(0.98)
atm_corr = lst.map(lambda image: image.multiply(0.99).add(0.11).multiply(emissivity).subtract(2.5))
wst = atm_corr.map(lambda image: image.subtract(273.15).rename('wst'))

# Redefine s2_dataset by adding computed indices directly to each image
s2_dataset = s2_dataset.map(lambda image: 
    image.addBands([
        image.normalizedDifference(ndwi_bands).rename('ndwi'),
        image.normalizedDifference(ndci_bands).rename('ndci'),
        image.normalizedDifference(ndti_bands).rename('ndti'),
        ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('ph'),
        image.select('B3').divide(0.1).divide(image.select('B2').divide(0.1)).pow(2.72).multiply(10.8).rename('chl_a'),
        ee.Image(0.0113).multiply(image.normalizedDifference(ndwi_bands).pow(3))
            .subtract(ee.Image(0.0135).multiply(image.normalizedDifference(ndwi_bands).pow(2)))
            .add(ee.Image(0.0075).multiply(image.normalizedDifference(ndwi_bands)))
            .add(ee.Image(2.5823)).rename('ssc'),
        image.select('B11').divide(10).add(273.15).divide(
            ee.Image(1).divide(image.select('B11').divide(14380).add(1).log())
        ).multiply(0.99).add(0.11).multiply(emissivity).subtract(2.5).subtract(273.15).rename('wst')
    ])
)




In [12]:
# Function to get the start dates with a weekly interval
def get_dates(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    dates = []
    while start <= end:
        dates.append(start.strftime('%Y-%m-%d'))
        start += timedelta(days=7)
    return dates


In [13]:
# Define the function to compute weekly averages for a given band
def compute_weekly_averages(dataset, band_name, start_date, end_date, output_file):
    # Define the date range
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    
    # Generate a list of weeks using get_dates function
    weeks = get_dates(start_date, end_date)

    all_stats = []

    for week_start in weeks:
        week_start = ee.Date(week_start)
        week_end = week_start.advance(1, 'week')
        
        # Filter the dataset for the week and select the band
        weekly_data = dataset.filterDate(week_start, week_end).select(band_name).mean()
        
        # Ensure the image has the band
        if weekly_data.bandNames().size().getInfo() == 0:
            print(f"No data for week starting {week_start.format('YYYY-MM-dd').getInfo()}")
            continue
        
        # Compute the zonal statistics
        stats = geemap.zonal_stats(
            weekly_data, 
            lake, 
            None,  # No need to save intermediate files
            stat_type='MEAN', 
            scale=1000, 
            return_fc=True
        )
        
        # Convert the results to a DataFrame and add the week column
        df = geemap.ee_to_df(stats)
        df['week_start'] = week_start.format('YYYY-MM-dd').getInfo()
        all_stats.append(df)
    
    # Concatenate all weekly DataFrames into a single DataFrame
    final_df = pd.concat(all_stats, ignore_index=True)
    
    # Save the final DataFrame to a CSV file
    final_df.to_csv(output_file, index=False)
    
    return output_file

# Define the function to compute weekly NDWI averages
def compute_weekly_ndwi_averages(start_date, end_date):
    ndwi_bands = ['B3', 'B8']
    ndwi_dataset = s2_dataset.map(lambda image: image.normalizedDifference(ndwi_bands).rename('ndwi'))
    output_file = 'ndwi_weekly.csv'
    return compute_weekly_averages(ndwi_dataset, 'ndwi', start_date, end_date, output_file)

# Define the function to compute weekly NDCI averages
def compute_weekly_ndci_averages(start_date, end_date):
    ndci_bands = ['B5', 'B4']
    ndci_dataset = s2_dataset.map(lambda image: image.normalizedDifference(ndci_bands).rename('ndci'))
    output_file = 'ndci_weekly.csv'
    return compute_weekly_averages(ndci_dataset, 'ndci', start_date, end_date, output_file)

# Define the function to compute weekly NDTI averages
def compute_weekly_ndti_averages(start_date, end_date):
    ndti_bands = ['B5', 'B11']
    ndti_dataset = s2_dataset.map(lambda image: image.normalizedDifference(ndti_bands).rename('ndti'))
    output_file = 'ndti_weekly.csv'
    return compute_weekly_averages(ndti_dataset, 'ndti', start_date, end_date, output_file)

# Define the function to compute weekly pH averages
def compute_weekly_ph_averages(start_date, end_date):
    ph_dataset = s2_dataset.map(lambda image: ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('ph'))
    output_file = 'ph_weekly.csv'
    return compute_weekly_averages(ph_dataset, 'ph', start_date, end_date, output_file)

# Define the function to compute weekly chlorophyll-a averages
def compute_weekly_chl_a_averages(start_date, end_date):
    chl_a_dataset = s2_dataset.map(lambda image: image.select('B3').divide(0.1).divide(image.select('B2').divide(0.1)).pow(2.72).multiply(10.8).rename('chl_a'))
    output_file = 'chl_a_weekly.csv'
    return compute_weekly_averages(chl_a_dataset, 'chl_a', start_date, end_date, output_file)

# Define the function to compute weekly SSC averages
def compute_weekly_ssc_averages(start_date, end_date):
    ssc_dataset = s2_dataset.map(lambda image: ee.Image(0.0113).multiply(image.select('ndwi').pow(3))
                                .subtract(ee.Image(0.0135).multiply(image.select('ndwi').pow(2)))
                                .add(ee.Image(0.0075).multiply(image.select('ndwi')))
                                .add(ee.Image(2.5823)).rename('ssc'))
    output_file = 'ssc_weekly.csv'
    return compute_weekly_averages(ssc_dataset, 'ssc', start_date, end_date, output_file)

In [ ]:
#Compute weekly NDCI averages
ndci_output_file = compute_weekly_ndci_averages('2017-03-28', '2024-11-12')
print(f'Weekly NDCI averages saved to {ndci_output_file}')

In [ ]:
# # Compute the weekly averages for each band
# ndwi_output_file = compute_weekly_ndwi_averages(start_date, end_date)
# ndci_output_file = compute_weekly_ndci_averages(start_date, end_date)
# ndti_output_file = compute_weekly_ndti_averages(start_date, end_date)
# ph_output_file = compute_weekly_ph_averages(start_date, end_date)
# chl_a_output_file = compute_weekly_chl_a_averages(start_date, end_date)
# ssc_output_file = compute_weekly_ssc_averages(start_date, end_date)

In [ ]:

# print(f'Weekly NDWI averages saved to {ndwi_output_file}')
# print(f'Weekly NDCI averages saved to {ndci_output_file}')
# print(f'Weekly NDTI averages saved to {ndti_output_file}')
# print(f'Weekly pH averages saved to {ph_output_file}')
# print(f'Weekly chlorophyll-a averages saved to {chl_a_output_file}')
# print(f'Weekly SSC averages saved to {ssc_output_file}')